In [66]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import pandas as pd
import io
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.svm import SVR


In [67]:
diamonds = pd.read_csv('../input/data.csv')
test = pd.read_csv('../input/test.csv')

In [68]:
diamonds.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,2.26,Ideal,G,SI2,61.9,57.0,8.44,8.36,5.20,12831
1,2.43,Very Good,H,SI2,63.2,57.0,8.56,8.50,5.39,16170
2,0.80,Premium,F,SI2,61.0,57.0,6.03,6.01,3.67,2797
3,0.40,Ideal,F,I1,63.3,60.0,4.68,4.64,2.95,630
4,0.31,Ideal,G,VS2,61.6,55.0,4.39,4.37,2.70,698


In [69]:
def cutOrdinal(a):
    if a == 'Ideal':
        return 1
    elif a == 'Very Good':
        return 2
    elif a == 'Premium':
        return 3
    elif a == 'Good':
        return 4
    else:
        return 5

In [70]:
def colorOrdinal(a):
    if a == 'G':
        return 1
    elif a == 'H':
        return 2
    elif a == 'F':
        return 3
    elif a == 'D':
        return 4
    elif a == 'E':
        return 5
    elif a == 'I':
        return 6
    else:
        return 7

In [71]:
def clarityOrdinal(a):
    if a == 'SI2':
        return 1
    elif a == 'I2':
        return 2
    elif a == 'VS2':
        return 3
    elif a == 'SI1':
        return 4
    elif a == 'VS1':
        return 5
    elif a == 'VVS2':
        return 6
    elif a == 'IF':
        return 7
    else:
        return 8

In [72]:
diamonds['cut'] = diamonds['cut'].apply(cutOrdinal)
diamonds['color'] = diamonds['color'].apply(colorOrdinal)
diamonds['clarity'] = diamonds['clarity'].apply(clarityOrdinal)

In [73]:
diamonds.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,2.26,1,1,1,61.9,57.0,8.44,8.36,5.20,12831
1,2.43,2,2,1,63.2,57.0,8.56,8.50,5.39,16170
2,0.80,3,3,1,61.0,57.0,6.03,6.01,3.67,2797
3,0.40,1,3,8,63.3,60.0,4.68,4.64,2.95,630
4,0.31,1,1,3,61.6,55.0,4.39,4.37,2.70,698


In [74]:
corrMatrix = diamonds.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,carat,cut,color,clarity,depth,table,x,y,z,price
carat,1,0.17,0.075,-0.29,0.024,0.18,0.98,0.94,0.95,0.92
cut,0.17,1,0.026,-0.14,0.13,0.52,0.17,0.15,0.17,0.086
color,0.075,0.026,1,-0.082,0.0049,0.043,0.062,0.061,0.063,0.012
clarity,-0.29,-0.14,-0.082,1,-0.011,-0.12,-0.31,-0.29,-0.3,-0.15
depth,0.024,0.13,0.0049,-0.011,1,-0.3,-0.029,-0.033,0.091,-0.014
table,0.18,0.52,0.043,-0.12,-0.3,1,0.2,0.18,0.15,0.13
x,0.98,0.17,0.062,-0.31,-0.029,0.2,1,0.97,0.97,0.88
y,0.94,0.15,0.061,-0.29,-0.033,0.18,0.97,1,0.94,0.86
z,0.95,0.17,0.063,-0.3,0.091,0.15,0.97,0.94,1,0.86
price,0.92,0.086,0.012,-0.15,-0.014,0.13,0.88,0.86,0.86,1


In [75]:
columns = ['depth', 'table']
for col in columns:
    diamonds[col] = (diamonds[col] - np.mean(diamonds[col])) / np.std(diamonds[col]) 

diamonds.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,2.26,1,1,1,0.106170,-0.205764,8.44,8.36,5.20,12831
1,2.43,2,2,1,1.013798,-0.205764,8.56,8.50,5.39,16170
2,0.80,3,3,1,-0.522188,-0.205764,6.03,6.01,3.67,2797
3,0.40,1,3,8,1.083616,1.138849,4.68,4.64,2.95,630
4,0.31,1,1,3,-0.103283,-1.102173,4.39,4.37,2.70,698


In [76]:
'''upper = corrMatrix.where(np.triu(np.ones(corrMatrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.97)]
to_drop'''

'upper = corrMatrix.where(np.triu(np.ones(corrMatrix.shape), k=1).astype(np.bool))\nto_drop = [column for column in upper.columns if any(upper[column] > 0.97)]\nto_drop'

In [77]:
diamonds = diamonds.drop(['x', 'y', 'z'], axis=1)
diamonds.head()

,carat,cut,color,clarity,depth,table,price
0,2.26,1,1,1,0.106170,-0.205764,12831
1,2.43,2,2,1,1.013798,-0.205764,16170
2,0.80,3,3,1,-0.522188,-0.205764,2797
3,0.40,1,3,8,1.083616,1.138849,630
4,0.31,1,1,3,-0.103283,-1.102173,698


In [78]:
X = diamonds[['carat', 'cut', 'color', 'clarity', 'depth', 'table']]
y = diamonds['price']

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [82]:
models = {
    #"svm": LinearSVC(),
    #"logistic": LogisticRegression(solver='lbfgs', max_iter=2000),
    "forest": RandomForestRegressor(n_trees = 100),
    #"ridge": Ridge(),
    #"svr": SVR(C=1.0, epsilon=0.2),
}

for modelName, model in models.items():
    print(f"Training model: {modelName}")
    model.fit(X_train, y_train)
    print(f"Training finished: {modelName}")

TypeError: __init__() got an unexpected keyword argument 'n_trees'

In [81]:

kf = KFold(n_splits=2, shuffle=True, random_state=11)
scores = cross_val_score(models['forest'], X_train, y_train, scoring='r2', cv=kf, n_jobs=2)
name = str(models['forest'])

print(f'{name}: Average R-Squared Score:', np.mean(scores))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False): Average R-Squared Score: 0.962801988238


In [63]:
test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.56,Ideal,I,VS2,62.1,54.0,5.30,5.33,3.30
1,1,1.05,Ideal,G,VS2,61.9,56.0,6.56,6.52,4.05
2,2,0.50,Premium,E,VS2,61.5,56.0,5.11,5.07,3.13
3,3,0.80,Ideal,F,VS1,62.8,56.0,5.91,5.96,3.73
4,4,0.54,Ideal,G,VS1,61.4,57.0,5.23,5.20,3.20


In [64]:
test['cut'] = test['cut'].apply(cutOrdinal)
test['color'] = test['color'].apply(colorOrdinal)
test['clarity'] = test['clarity'].apply(clarityOrdinal)
test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.56,1,6,3,62.1,54.0,5.30,5.33,3.30
1,1,1.05,1,1,3,61.9,56.0,6.56,6.52,4.05
2,2,0.50,3,5,3,61.5,56.0,5.11,5.07,3.13
3,3,0.80,1,3,5,62.8,56.0,5.91,5.96,3.73
4,4,0.54,1,1,5,61.4,57.0,5.23,5.20,3.20


In [65]:

d1 = {modelName:model.predict(test[["carat", "cut", "color", "clarity", "depth", "table"]]) for modelName, model in models.items()}

df1 = pd.DataFrame(d1)
df1['id']=test['id']
new = df1[['id','forest']]
new.rename(columns={"forest": "price"}, inplace=True)
new.head()

,id,price
0,0,1303.0
1,1,6559.3
2,2,1387.2
3,3,3689.0
4,4,1592.4
